# Scraping de Natural by Lila

Nuestro objetivo consiste en escrapear la pagina de vestidos de natural by Lila por categorías, por lo que usaremos tanto Selenium como Beautiful Soup.

In [1]:
import sys
import os
import pandas as pd
import json

sys.path.append(os.path.abspath('../'))
from src import funciones_scraping as fs

In [8]:
lista_vestidos_cortos_largos = fs.selenium_bylila()

In [14]:
with open("../datos/scraping/links_nbl.json", "w") as archivo_json:
    json.dump(lista_vestidos_cortos_largos, archivo_json)

### Procedemos a llevar a cabo la extracción de los datos de nuestro interés.

In [3]:
with open("../datos/scraping/links_nbl.json", "r") as archivo_json:
    lista_vestidos_cortos_largos = json.load(archivo_json)

In [7]:
dic_vestido = fs.extraccion_info(lista_vestidos_cortos_largos)

In [13]:
df_vestidos_mbl = pd.DataFrame(dic_vestido)

En el caso de esta tiendo como no se puede estudiar la disponibilidad ni el stock por prendas de distintos colores, la columna de color no estará presenta en este dataframe ya que no aporta ningún valor.

Una vez comprobado que no hay duplicados ni nulos almacenamos el dataframe.

In [11]:
df_vestidos_mbl.to_csv("../datos/dataframes/df_bylila.csv")

In [12]:
df_vestidos_mbl.head(3)

,nombre,marca,precio,talla,categoria
0,Mini Vestido Strapless Cascada,Natural by Lila,49.99,S,corto
1,Mini Vestido Strapless Cascada,Natural by Lila,49.99,M,corto
2,Vestido Asimétrico Choker Lurex,Natural by Lila,39.99,S,corto
